In [1]:
import findspark
findspark.init()

In [2]:
from time import time

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext()

#### 1. Đọc dữ liệu từ file "kddcup.data_10_percent.gz". Tạo RDD từ dữ liệu này

In [4]:
file = "../../Data/kddcup.data_10_percent.gz"
data = sc.textFile(file)

#### 2. Cho biết dữ liệu có bao nhiêu element

In [5]:
print("Lines:", data.count())

Lines: 494021


#### 3. In thông tin 3 element đầu tiên của RDD

In [6]:
data.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

In [7]:
data.first()

'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.'

#### 4. Tạo một RDD mới từ RDD trên, với điều kiện các element trong RDD mới đều có chuỗi "normal"

In [8]:
normal_data = data.filter(lambda x: 'normal.' in x)

#### 5. Cho biết RDD mới có bao nhiêu element, mất bao nhiêu thời gian để đếm số element này?

In [9]:
t0 = time()
normal_count = normal_data.count()
t1 = time() - t0

print("There are", normal_count, "'normal' in s")
print("Time to count:", format(round(t1, 3)), "seconds")

There are 97278 'normal' in s
Time to count: 1.496 seconds


#### 6. Tạo array chứa tất cả các element của RDD ban đầu, mất bao nhiêu thời gian để thực hiện việc này?

In [10]:
t0 = time()
array_data = data.collect()
t1 = time() - t0

print("Data collected in:", format(round(t1, 3)), "seconds")

Data collected in: 3.272 seconds


#### 7. Tạo một RDD sample bằng 1/10 RDD ban đầu, các element lấy ngẫu nhiên

In [11]:
# param 1: the sampling is done with replacement or not
# param 2: the sample size as fraction
# param 3: [optionally] provide a random seed
data_sample = data.sample(False, 0.1, 42)
sample_size = data_sample.count()
total_size = data.count()

print("Sample size is", sample_size, "of", total_size)

Sample size is 49387 of 494021


#### 8. Cho biết tỉ lệ các element chứa chuỗi "normal." so với tổng số element trong RDD sample, mất bao nhiêu thời gian để thực hiện việc này?

In [12]:
sample_normal_tags = data_sample.filter(lambda x: 'normal.' in x)

t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
t1 = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("Sample with normal size is", sample_normal_tags_count)
print("The ratio of 'normal' interactions is", round(sample_normal_ratio, 3))
print("Count done in", round(t1, 3), "seconds")

Sample with normal size is 9733
The ratio of 'normal' interactions is 0.197
Count done in 1.804 seconds


#### 9. Tạo RDD sample với các element không chứa từ 'normal.' từ RDD sample trên. Số element của RDD này là bao nhiêu? In thông tin element đầu tiên của RDD này

In [13]:
sample_without_normal = data_sample.subtract(sample_normal_tags)
print("Sample without normal size is", sample_without_normal.count())

Sample without normal size is 39654


In [14]:
sample_without_normal.take(1)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,509,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.']

#### 10. Với RDD ban đầu, mỗi element là một chuỗi. Hãy tạo ra một RDD mới (array_RDD) với mỗi element là một danh sách các phần tử được tách ra từ một chuỗi với dấu phân cách là ",". In thông tin element đầu tiên của RDD này

In [15]:
array_RDD = data.map(lambda x: x.split(","))
array_RDD.first()

['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']

#### 11. Với array_Rdd trên, hãy cho biết có những protocol nào được sử dụng? Có bao nhiêu protocol? (Biết protocol là phần tử thứ hai trong từng element)

In [16]:
protocols = array_RDD.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'udp', 'icmp']

In [17]:
print("Protocols:", protocols.count())

Protocols: 3


#### 12. Với array_Rdd trên, hãy cho biết có những service nào được sử dụng? Có bao nhiêu service? (Biết service là phần tử thứ ba trong từng element)

In [18]:
services = array_RDD.map(lambda x: x[2]).distinct()
services.collect()

['http',
 'smtp',
 'finger',
 'domain_u',
 'auth',
 'telnet',
 'ftp',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'rje',
 'time',
 'mtp',
 'link',
 'remote_job',
 'gopher',
 'ssh',
 'name',
 'whois',
 'domain',
 'login',
 'imap4',
 'daytime',
 'ctf',
 'nntp',
 'shell',
 'IRC',
 'nnsp',
 'http_443',
 'exec',
 'printer',
 'efs',
 'courier',
 'uucp',
 'klogin',
 'kshell',
 'echo',
 'discard',
 'systat',
 'supdup',
 'iso_tsap',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'sql_net',
 'vmnet',
 'bgp',
 'Z39_50',
 'ldap',
 'netstat',
 'urh_i',
 'X11',
 'urp_i',
 'pm_dump',
 'tftp_u',
 'tim_i',
 'red_i']

In [19]:
print("Services:", services.count())

Services: 66


#### 13. Một sản phẩm được tạo ra sẽ có một protocol và một service. Tạo một bảng danh sách các sản phẩm bằng cách phối hợp từng protocol và service của 2 câu trên (gợi ý: Dùng RDD1.cartesian(RDD2.collect()). In kết quả. Có bao nhiêu sản phẩm được tạo ra

In [20]:
product = protocols.cartesian(services).collect()
product

[('tcp', 'http'),
 ('tcp', 'smtp'),
 ('tcp', 'finger'),
 ('tcp', 'domain_u'),
 ('tcp', 'auth'),
 ('tcp', 'telnet'),
 ('tcp', 'ftp'),
 ('tcp', 'eco_i'),
 ('tcp', 'ntp_u'),
 ('tcp', 'ecr_i'),
 ('tcp', 'other'),
 ('tcp', 'private'),
 ('tcp', 'pop_3'),
 ('tcp', 'ftp_data'),
 ('tcp', 'rje'),
 ('tcp', 'time'),
 ('tcp', 'mtp'),
 ('tcp', 'link'),
 ('tcp', 'remote_job'),
 ('tcp', 'gopher'),
 ('tcp', 'ssh'),
 ('tcp', 'name'),
 ('tcp', 'whois'),
 ('tcp', 'domain'),
 ('tcp', 'login'),
 ('tcp', 'imap4'),
 ('tcp', 'daytime'),
 ('tcp', 'ctf'),
 ('tcp', 'nntp'),
 ('tcp', 'shell'),
 ('tcp', 'IRC'),
 ('tcp', 'nnsp'),
 ('tcp', 'http_443'),
 ('tcp', 'exec'),
 ('tcp', 'printer'),
 ('tcp', 'efs'),
 ('tcp', 'courier'),
 ('tcp', 'uucp'),
 ('tcp', 'klogin'),
 ('tcp', 'kshell'),
 ('tcp', 'echo'),
 ('tcp', 'discard'),
 ('tcp', 'systat'),
 ('tcp', 'supdup'),
 ('tcp', 'iso_tsap'),
 ('tcp', 'hostnames'),
 ('tcp', 'csnet_ns'),
 ('tcp', 'pop_2'),
 ('tcp', 'sunrpc'),
 ('tcp', 'uucp_path'),
 ('tcp', 'netbios_ns'),
 ('t

In [21]:
print("There are", len(product), "combination of protocol X service")

There are 198 combination of protocol X service


#### 14. Cho biết số lượng partition của array_RDD

In [22]:
array_RDD.getNumPartitions()

1

#### 15. Lưu array_RDD vào thư mục "kdd_cup" với mỗi partition là một file riêng lẻ

In [24]:
array_RDD.saveAsTextFile("kdd_cup")